# Farnebäck Flow

In [1]:
# init prerequisites
%reload_ext autoreload
%autoreload 2
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os, re, math, time, random
from pprint import pprint

import util
from util import print_warn, horline

print("OpenCV {}".format(cv.__version__))
print("Numpy {}".format(np.__version__))

print_warn("Warning Text!")
horline()


OpenCV 3.4.1
Numpy 1.16.2
 Warning Text! 
________________________________________________________________________________________________________________________


In [2]:
# gasonics production equipment
config = dict(
    inputfolder=r".\input\classes_r2",
    outputfolder=r".\output\flow",  
)
pprint(config)

{'inputfolder': '.\\input\\classes_r2', 'outputfolder': '.\\output\\flow'}


In [3]:
# collect input files
paths, labels, label_mapping = util.recursive_collect(config["inputfolder"])
input_images = []
prev = None

shuffle = list(zip(paths, labels))
# shuffle or sort
#random.shuffle(shuffle)
shuffle = sorted(shuffle, key = lambda x: x[0].split("\\")[-1])
paths, labels = zip(*shuffle[300:])

# one time initialization
orb = cv.ORB_create(nfeatures=1500, scaleFactor=2, patchSize=31, edgeThreshold=31)
clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)

# read input image and store for future use
robot_light = cv.imread("img_gasonics/robot_light.png", cv.IMREAD_GRAYSCALE)

# find keypoints
robot_light_kp = orb.detect(robot_light,None)
# compute the descriptors
robot_light_kp, robot_light_des = orb.compute(robot_light, robot_light_kp)
print("  found {} keypoints for search image".format(len(robot_light_kp)))

cv.destroyAllWindows()
out = None
for i, path, label in zip(range(len(paths)), paths, labels):
    print(i, path, label)
    start_time = time.clock()
    
    # read input image and store for future use
    img = cv.imread(path, cv.IMREAD_COLOR)
    img = cv.resize(img, (int(img.shape[1]*0.5), int(img.shape[0]*0.5)))
    img = cv.medianBlur(img, 13)
    cv.imshow('img', img)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    if prev is None:
        prev = gray
        hsv = np.zeros_like(img)
        hsv[:,:,1] = 255
    
#    print(prev.shape, gray.shape)
#    print(hsv.shape)
        
    flow = cv.calcOpticalFlowFarneback(prev,gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv.cartToPolar(flow[:,:,0], flow[:,:,1])
    hsv[:,:,0] = ang*180/np.pi/2
    hsv[:,:,2] = cv.normalize(mag,None,0,255,cv.NORM_MINMAX)
    bgr = cv.cvtColor(hsv,cv.COLOR_HSV2BGR)
    cv.imshow('flow', bgr)
    
    if out is None:
        outh,outw,_ = bgr.shape
        out = cv.VideoWriter(os.path.join(config["outputfolder"], "farnebäck-orb.avi"), cv.VideoWriter_fourcc(*'MJPG'), 10, (outw,outh))
    vidimg = cv.addWeighted(bgr, 0.5, img, 0.5, 0)
    cv.imshow('save', vidimg)
    out.write(vidimg)
    prev = gray
    
    #img = cv.fastNlMeansDenoisingColored(img,None,10,10,7,21)
    #img = cv.equalizeHist(img)
    #img = clahe.apply(img)
    #img = cv.adaptiveThreshold(img, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 127, 2)
    #input_images.append(img)
    
    # find keypoints
    kp = orb.detect(img,None)
    
    # compute the descriptors
    kp, des = orb.compute(img, kp)
    #print("  found {} keypoints".format(len(kp)))
    
    #matches = bf.match(robot_light_des,des)
    #matches = sorted(matches, key = lambda x:x.distance)
    #img3 = cv.drawMatches(robot_light,robot_light_kp,img,kp,matches[:10],img,flags=2)
    #plt.imshow(img3),plt.show()
    
    # show keypoint location
    img2 = cv.drawKeypoints(img, kp, None, color=(0,255,0), flags=4)
    end_time = time.clock()
    print("  computation for {} took {:.2f}ms".format(path, (end_time - start_time)*1000.0))
    
    #plt.subplot(1,3,3)
    #plt.imshow(img2), plt.show()
    cv.imshow('keypoints', img2)
    k = cv.waitKey(20) & 0xff
    if k == 27:
        if out is not None:
            out.release()
        break
cv.destroyAllWindows()

  found 6 keypoints for search image
0 .\input\classes_r2\4.2_processchamber_unknown\gray_89010.0.png 8
  computation for .\input\classes_r2\4.2_processchamber_unknown\gray_89010.0.png took 116.96ms
1 .\input\classes_r2\4.2_processchamber_unknown\gray_89011.0.png 8


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


  computation for .\input\classes_r2\4.2_processchamber_unknown\gray_89011.0.png took 76.96ms
2 .\input\classes_r2\4.2_processchamber_unknown\gray_89012.0.png 8
  computation for .\input\classes_r2\4.2_processchamber_unknown\gray_89012.0.png took 76.36ms
3 .\input\classes_r2\4.2_processchamber_unknown\gray_89013.0.png 8
  computation for .\input\classes_r2\4.2_processchamber_unknown\gray_89013.0.png took 74.86ms
4 .\input\classes_r2\4.2_processchamber_unknown\gray_89014.0.png 8
  computation for .\input\classes_r2\4.2_processchamber_unknown\gray_89014.0.png took 76.43ms
5 .\input\classes_r2\4.3_processchamber_wafer\gray_89015.0.png 9
  computation for .\input\classes_r2\4.3_processchamber_wafer\gray_89015.0.png took 74.74ms
6 .\input\classes_r2\4.3_processchamber_wafer\gray_89016.0.png 9
  computation for .\input\classes_r2\4.3_processchamber_wafer\gray_89016.0.png took 76.11ms
7 .\input\classes_r2\4.3_processchamber_wafer\gray_89017.0.png 9
  computation for .\input\classes_r2\4.3_pro

  computation for .\input\classes_r2\3.2_cassette_location_unknown\gray_89064.0.png took 75.50ms
55 .\input\classes_r2\3.2_cassette_location_unknown\gray_89065.0.png 5
  computation for .\input\classes_r2\3.2_cassette_location_unknown\gray_89065.0.png took 75.78ms
56 .\input\classes_r2\3.2_cassette_location_unknown\gray_89066.0.png 5
  computation for .\input\classes_r2\3.2_cassette_location_unknown\gray_89066.0.png took 75.26ms
57 .\input\classes_r2\3.2_cassette_location_unknown\gray_89067.0.png 5
  computation for .\input\classes_r2\3.2_cassette_location_unknown\gray_89067.0.png took 75.72ms
58 .\input\classes_r2\3.2_cassette_location_unknown\gray_89068.0.png 5
  computation for .\input\classes_r2\3.2_cassette_location_unknown\gray_89068.0.png took 75.76ms
59 .\input\classes_r2\3.2_cassette_location_unknown\gray_89069.0.png 5
  computation for .\input\classes_r2\3.2_cassette_location_unknown\gray_89069.0.png took 77.77ms
60 .\input\classes_r2\3.2_cassette_location_unknown\gray_89070.

106 .\input\classes_r2\4.1_processchamber_empty\gray_89116.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89116.0.png took 75.78ms
107 .\input\classes_r2\4.1_processchamber_empty\gray_89117.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89117.0.png took 77.66ms
108 .\input\classes_r2\4.1_processchamber_empty\gray_89118.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89118.0.png took 76.61ms
109 .\input\classes_r2\4.1_processchamber_empty\gray_89119.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89119.0.png took 76.31ms
110 .\input\classes_r2\4.1_processchamber_empty\gray_89120.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89120.0.png took 77.50ms
111 .\input\classes_r2\4.1_processchamber_empty\gray_89121.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89121.0.png took 77.67ms
112 .\input\classes_r2\4.1_processchamber_empt

  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89349.0.png took 75.14ms
159 .\input\classes_r2\4.1_processchamber_empty\gray_89350.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89350.0.png took 76.54ms
160 .\input\classes_r2\4.1_processchamber_empty\gray_89351.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89351.0.png took 76.02ms
161 .\input\classes_r2\4.1_processchamber_empty\gray_89352.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89352.0.png took 77.18ms
162 .\input\classes_r2\4.1_processchamber_empty\gray_89353.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89353.0.png took 75.30ms
163 .\input\classes_r2\4.1_processchamber_empty\gray_89354.0.png 7
  computation for .\input\classes_r2\4.1_processchamber_empty\gray_89354.0.png took 77.02ms
164 .\input\classes_r2\4.1_processchamber_empty\gray_89355.0.png 7
  computation for .\input\classes_r2\4.1_proce

In [ ]:
cv.destroyAllWindows()